<a href="https://colab.research.google.com/github/UrienLin/NLPTest/blob/master/KBQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




## KBQA 
> [根據此篇文章](https://zhuanlan.zhihu.com/p/62946533)
### 模組1. NER 辨識 Question 中的 KeyWord
1-1. 利用關鍵字，標註關鍵字在 問句 的 index<BR> 
1-2. 將 1-1 做為TokenClassfication的LABEL位置<BR>
1-3. ~~訓練 NER 模型 (Input 問句 Output 關鍵字)~~<BR>

update 1-3 >> 改用HANLP <br>
update 1-3 >> 僅完成前100筆資料清整測試

---
1-4. 產生問答 Triple Set
### 模組2. 句子相似度
2-1. 將模組1.的 Output 關鍵字在 Knowledge 中搜尋<BR>
2-2. 將 2-1 的句子集合 與 問題 做句子相似度判斷 <BR>
2-3. 2-2 該項最相似的一筆資料的 ANSWER 作為回答



In [46]:
!pip install jieba -q
!pip install transformers -q
!pip install hanlp -q
from google.colab import drive
drive.mount('/content/drive')

import pandas as  pd
import jieba.analyse
import hanlp

tokenizer = hanlp.load('PKU_NAME_MERGED_SIX_MONTHS_CONVSEG')
tagger = hanlp.load(hanlp.pretrained.pos.CTB5_POS_RNN_FASTTEXT_ZH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Executing op HashTableV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op HashTableV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op HashTableV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op HashTableV2 in device /job:localhost/replica:0/task:0/device:CPU:0


In [0]:
#PATH
raw = pd.read_csv('/content/drive/My Drive/20190825 Challenge/04_Kbqa.training-data', sep='\t',header=None)
#LOCAL PATH
#raw = pd.read_csv('./04_Kbqa.training-data', sep='\t',header=None)

In [0]:
raw.columns = ['iden','text']
_df = raw.head(100)
_df['token'] = _df.text.apply(lambda x: tokenizer(str(x)))
_df['POS'] = _df['token'].apply(lambda x : tagger(x))

In [52]:
_df.head()

,iden,text,token,POS,POS_ID
0,<question id=1>,《机械设计基础》这本书的作者是谁？,"[《, 机械, 设计, 基础, 》, 这, 本, 书, 的, 作者, 是, 谁, ？]","[PU, NN, NN, NN, PU, DT, M, NN, DEG, NN, VC, P...","[0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0]"
1,<answer id=1>,杨可桢，程光蕴，李仲生,"[杨可桢, ，, 程光蕴, ，, 李仲生]","[NR, PU, NR, PU, NR]","[0, 0, 0, 0, 0]"
2,==============================================...,NaN,[nan],[NN],[1]
3,<question id=2>,《高等数学》是哪个出版社出版的？,"[《, 高等, 数学, 》, 是, 哪个, 出版社, 出版, 的, ？]","[PU, JJ, NN, PU, VC, DT, NN, VV, DEC, PU]","[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]"
4,<answer id=2>,武汉大学出版社,"[武汉, 大学, 出版社]","[NR, NN, NN]","[0, 1, 1]"


In [50]:
def POS_Weight(list_POS):
  return ['1' if x=='NN' else '0' for x in list_POS]

_df['POS_ID'] = _df.POS.apply(POS_Weight)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [51]:
_df

,iden,text,token,POS,POS_ID
0,<question id=1>,《机械设计基础》这本书的作者是谁？,"[《, 机械, 设计, 基础, 》, 这, 本, 书, 的, 作者, 是, 谁, ？]","[PU, NN, NN, NN, PU, DT, M, NN, DEG, NN, VC, P...","[0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0]"
1,<answer id=1>,杨可桢，程光蕴，李仲生,"[杨可桢, ，, 程光蕴, ，, 李仲生]","[NR, PU, NR, PU, NR]","[0, 0, 0, 0, 0]"
2,==============================================...,NaN,[nan],[NN],[1]
3,<question id=2>,《高等数学》是哪个出版社出版的？,"[《, 高等, 数学, 》, 是, 哪个, 出版社, 出版, 的, ？]","[PU, JJ, NN, PU, VC, DT, NN, VV, DEC, PU]","[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]"
4,<answer id=2>,武汉大学出版社,"[武汉, 大学, 出版社]","[NR, NN, NN]","[0, 1, 1]"
...,...,...,...,...,...
95,==============================================...,NaN,[nan],[NN],[1]
96,<question id=33>,东创建国主要做什么生意,"[东创, 建国, 主要, 做, 什么, 生意]","[NR, NN, AD, VV, DT, NN]","[0, 1, 0, 0, 0, 1]"
97,<answer id=33>,整车销售、售后服务、配件供应,"[整车, 销售, 、, 售后服务, 、, 配件, 供应]","[NN, NN, PU, NN, PU, NN, NN]","[1, 1, 0, 1, 0, 1, 1]"
98,==============================================...,NaN,[nan],[NN],[1]
